In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast
import time
today = time.strftime("%Y_%m_%d")
from enrich_dimensions.rounds import rounds, rounding_inch_feet,rounding_lbs,rounding_w,rounding_oz, rounding_lb,rounding_gal, re_extract, curate, round_string_float, clean_data,reg_ex,rounding_period_after_unit 
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

def get_unique_list(material_list):
    unique_list = []
    for item in material_list:
        if isinstance(item, str) and "[" in item and "]" in item:
            item_values = ast.literal_eval(item)
            unique_list.extend(item_values)
        else:
            unique_list.append(item)
    return sorted(list(set(unique_list)))

customer_id = '66'
customer_name='%bjs%'
client='BJ'

In [2]:
attribut = 'quantity'
dateszs='2001-08-11'
dateszs='2001-11-01'
curation_col = f'Q:{attribut}'
params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs,
         'attribut':attribut}

print('Start Query of Curated Data')
cb = query_from_file(file_name='query/curated_all_attributes_date_family.sql', params=params)
print(len(cb))

fill_dict = cb.groupby('external_id')['buckets'].last().to_dict()
cb['buckets'] = cb['buckets'].fillna(cb['external_id'].map(fill_dict))

Start Query of Curated Data
80941


# Check Bucket Attribute Pairs For Attributes with Values

In [8]:
print(len(sorted(cb['attribute'].explode().value_counts().reset_index()['index'].to_list())))
lst_attribute=sorted(cb['attribute'].explode().value_counts().reset_index()['index'].to_list())
print('Number of attributes: '+str(len(lst_attribute)))
print('')

# lst_attribute=['concern']
for i in range(len(lst_attribute)):
    attri=lst_attribute[i]
    dateszs='2001-08-11'
    attribut=attri
    dateszs='2001-11-01'
    curation_col = f'Q:{attribut}'
    params = {'customer_id': customer_id,
              'customer_name':customer_name,
             'dateszs':dateszs,
             'attribut':attribut}

    print('Start Buckets')
    print(str(i)+' Attribute: '+str(attri))
    bucket_value = query_from_file(file_name='query/Bucket_Value_Strategy.sql', params=params)

    brand=cb[(cb['value'].astype(str)!='["n/a"]')&(cb['value'].astype(str)!='n/a')&(cb['attribute'].astype(str)==attri)]
    print('Number of SKUs for the attribute '+str(attri)+': '+str(len(brand)))
    list_of_all_buckets=sorted(list(set(bucket_value['buckets'].to_list())))

    brand=cb[(cb['value'].astype(str)!='["n/a"]')&(cb['value'].astype(str)!='n/a')&(cb['attribute'].astype(str)==attri)]
#     print('Number of SKUs for the attribute '+str(attri)+': '+str(len(brand)))
    list_of_all_buckets=sorted(list(set(bucket_value['buckets'].to_list())))

    filtered_dfs = {}
    b_lst=[]

    for i in range(len(list_of_all_buckets)):

        # values that are supposed to be in the buckets
        action_bucket_values=bucket_value[bucket_value['buckets'].astype(str)==list_of_all_buckets[i]]['values'].to_list()

        # values that are actually in the curation
        action=brand[brand['buckets'].astype(str)==list_of_all_buckets[i]]['value'].to_list()

        unique_values=get_unique_list(action)

        target=[x for x in unique_values if x not in action_bucket_values]
        if len(target) > 0:
            target = target[0]
            pat=f'''(?i)({target})|()'''
            bucket_filtered_df=brand[brand['buckets'].astype(str)==list_of_all_buckets[i]]
            bucket_filtered_df['match']=bucket_filtered_df['value'].apply(lambda x: re_extract(pat,str(x)))
            fil=bucket_filtered_df[bucket_filtered_df['match'].astype(str)!='[]']
            filtered_dfs[list_of_all_buckets[i]] = fil
            b_lst.append(list_of_all_buckets[i])
        else:
            pass

    
    b_lst[0:10]
    valuez = {}
    external_id_list=[]
    for i in range(len(b_lst)):
        valuez[b_lst[i]]=filtered_dfs[b_lst[i]][['external_id','value']]
        external_id_list.append(filtered_dfs[b_lst[i]]['external_id'].tolist())
    import itertools
    flattened_list = list(itertools.chain.from_iterable(external_id_list))
    if len(flattened_list)>0:
        print('Buckets Effected: '+str(len(b_lst)))
        df_name = f'match_{attri}'
        data = {'external_id': flattened_list, f'Q:{attri}': '[]'}
        df_dict = {df_name: pd.DataFrame(data)}
        new_df = df_dict[df_name]
        print('Number of SKUs to be wiped: '+str(len(new_df)))
        print('')
        print('')

        def get_df_name(df):
            name =[x for x in globals() if globals()[x] is df][0]
            return name
        def looks_good(customer, matches,attri): 
            drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
            matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_{attri}-{today}.csv',index=False) 
        looks_good(client, new_df,attri) 
    else:
        pass

102
Number of attributes: 102

Start Buckets
0 Attribute: activity
Number of SKUs for the attribute activity: 4
Start Buckets
1 Attribute: band_color
Number of SKUs for the attribute band_color: 95
Start Buckets
2 Attribute: battery_size
Number of SKUs for the attribute battery_size: 19
Start Buckets
3 Attribute: bed_size
Number of SKUs for the attribute bed_size: 641
Start Buckets
4 Attribute: botanical_type
Number of SKUs for the attribute botanical_type: 0
Start Buckets
5 Attribute: bottle_count
Number of SKUs for the attribute bottle_count: 6
Start Buckets
6 Attribute: bracelet_style
Number of SKUs for the attribute bracelet_style: 73
Start Buckets
7 Attribute: btu_s
Number of SKUs for the attribute btu_s: 37
Start Buckets
8 Attribute: bulb_type
Number of SKUs for the attribute bulb_type: 74
Start Buckets
9 Attribute: camera_count
Number of SKUs for the attribute camera_count: 11
Start Buckets
10 Attribute: can_count
Number of SKUs for the attribute can_count: 12
Start Buckets
11 A

# Check Bucket Attribute Pairs For Freetext Attributes

In [7]:
patterns = {
    'carat_total_weight': r'(ct\. t\.w\.)|(n\/a)|()',
    'btu_s': r'(\d(?!\,)(?!\d)(?! \w))|(n\/a)|()',
    'bottle_count': r'(\d(?!\,)(?!\d)(?! \w))|(n\/a)|()',
    'can_count':r'(\d(?!\,)(?!\d)(?! \w))|(n\/a)',
    'cubic_feet':r'(\d cu\. ft\.)|(n\/a)',
    'depth':r'(\d in\.)|(\d ft\.)|(n\/a)',
    'diameter':r'(\d in\.)|(\d ft\.)|(n\/a)',
    'lumens':r'(\d(?!\,)(?!\d)(?! \w))|(n\/a)|()',
    'length':r'(\d in\.)|(\d ft\.)|(n\/a)',
    'maximum_rpm':r'(\d+ RPM)|(n\/a)',
    'maximum_speed':r'(\d+ MPH)',
    'maximum_television_size':r'(\d+ in\.)',
    'number_of_resistance_levels':r'(\d(?!\,)(?!\d)(?! \w))|(n\/a)|()',
    'range':r'(\d in)|(n\/a)',
    'roll_length':r'(\d in)|(n\/a)',
    'screen_size':r'(\d in\.)|(n\/a)',
    'sheet_capacity': r'(\d(?!\,)(?!\d)(?! \w))|(n\/a)|()',
    'spf': r'(\d(?!\,)(?!\d)(?! \w))|(n\/a)|()',
    'voltage':r'(\d V)|(n\/a)',
    'wattage':r'(\dW)|(n\/a)',
    'weight':r'(\d lbs)|(n\/a)',
    'weight_capacity':r'(\d lbs)|(n\/a)',
    'width':r'(\d in\.)|(n\/a)'
}


In [4]:
def filter_empty_matches(dataframe, attribute, pattern):
    matches = dataframe[dataframe['attribute'].astype(str) == attribute]['value'].apply(lambda x: re_extract(pattern,str(x)))
    empty_matches = matches.apply(lambda x: len(x) == 0)
    return dataframe[dataframe['attribute'].astype(str) == attribute][empty_matches]

def count_empty_matches(dataframe, attribute, pattern):
    print(f'Number of SKUs for {attribute}: '+str(len(dataframe[dataframe['attribute'].astype(str) == attribute])))
    matches = dataframe[dataframe['attribute'].astype(str) == attribute]['value'].apply(lambda x: re_extract(pattern,str(x)))
    return sum(len(match) == 0 for match in matches)


filtered_dataframes = {}
for attribute, pattern in patterns.items():
    filtered_dataframes[attribute] = filter_empty_matches(cb, attribute, pattern)
    
counts = {}
for attribute, pattern in patterns.items():
    counts[attribute] = count_empty_matches(cb, attribute, pattern)
#     print(f'Rows that need to be wiped for {attribute}: '+f': {counts[attribute]}')
   

    try:
        flattened_list = filtered_dataframes[attribute]['external_id'].to_list()
        if len(flattened_list) > 0:
            df_name = f'match_{attribute}'
            data = {'external_id': flattened_list, f'Q:{attribute}': '[]'}
            df_dict = {df_name: pd.DataFrame(data)}
            new_df = df_dict[df_name]
            print('Number of SKUs to be wiped: ' + str(len(new_df)))
            print('')
            print('')

            def get_df_name(df):
                name = [x for x in globals() if globals()[x] is df][0]
                return name

            def looks_good(customer, matches, attribute):
                today = pd.Timestamp.today().strftime("%Y-%m-%d")
                drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload'
                matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_{attribute}-{today}.csv', index=False)
            looks_good(client, new_df, attribute)
            
        else:
            print('No SKUs to be wiped for ' + attribute)
    except KeyError:
        print('Error: DataFrame with name ' + df_name + ' does not exist')

Number of SKUs for carat_total_weight: 1223
No SKUs to be wiped for carat_total_weight
Number of SKUs for btu_s: 77
No SKUs to be wiped for btu_s
Number of SKUs for bottle_count: 28
No SKUs to be wiped for bottle_count
Number of SKUs for can_count: 24
No SKUs to be wiped for can_count
Number of SKUs for cubic_feet: 24
No SKUs to be wiped for cubic_feet
Number of SKUs for depth: 36
No SKUs to be wiped for depth
Number of SKUs for diameter: 699
No SKUs to be wiped for diameter
Number of SKUs for lumens: 28
No SKUs to be wiped for lumens
Number of SKUs for length: 953
Number of SKUs to be wiped: 11


Number of SKUs for maximum_rpm: 6
No SKUs to be wiped for maximum_rpm
Number of SKUs for maximum_speed: 0
Error: DataFrame with name match_length does not exist
Number of SKUs for maximum_television_size: 4
Number of SKUs to be wiped: 4


Number of SKUs for number_of_resistance_levels: 115
No SKUs to be wiped for number_of_resistance_levels
Number of SKUs for range: 4
No SKUs to be wiped for 

# Curated - All attributes

In [ ]:
attribut = 'quantity'
dateszs='2001-11-01'
curation_col = f'Q:{attribut}'
params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs,
         'attribut':attribut}

print('Start Query of Curated Data')
cb = query_from_file(file_name='query/curated_all_attributes_date_family.sql', params=params)
print(len(cb))

fill_dict = cb.groupby('external_id')['buckets'].last().to_dict()
cb['buckets'] = cb['buckets'].fillna(cb['external_id'].map(fill_dict))

In [17]:
cb[cb['attribute'].astype(str)=='depth_range']['value'].explode().value_counts()

12.1 - 24 in.    10
n/a               7
24.1 - 36 in.     2
0 - 6 in.         1
Name: value, dtype: int64

In [18]:
x_lst=cb[(cb['attribute'].astype(str)=='depth_range')&(cb['value'].astype(str)=='0 - 6 in.')]['external_id']

In [20]:
cb[(cb['attribute'].astype(str)=='depth')&(cb['external_id'].isin(x_lst))]

,attribute,buckets,value,customer_name,external_id,name,long_desc,custom_fields


In [3]:
cb['attribute'].explode().value_counts()

product_type                   11272
search_product_type             7659
quantity                        7335
color                           5728
concern                         4712
features                        4392
primary_flavor                  3354
material                        2844
supplement_type                 1735
metal_type                      1580
key_ingredient                  1513
height                          1501
pearl_type                      1485
stone_type                      1484
stone_shape                     1484
gold_karat                      1483
stone_color                     1483
finish                          1277
carat_total_weight              1223
height_range                    1159
length                           955
form                             942
indoor_outdoor                   837
fabric                           816
gender                           808
ring_style                       796
width                            796
r

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches,attri): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
    matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_{attri}-{today}.csv',index=False) 
looks_good(client, new_df,attri) 

# Uncurated

In [4]:
formatted_attribute = 'height_range'
buckets = """Office Chairs"""

# strategy_version_input=767
# attribute_id_input=2730

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'


df = query_from_file(file_name='./query/custom_fields.sql', params=params)
print(len(df))
print(today)

128
2023_06_06


In [9]:
height_lst=df['external_id'].to_list()
height_range=cb[(cb['attribute'].astype(str)=='height')&(cb['external_id'].isin(height_lst))]

In [10]:
height_range

,attribute,buckets,value,customer_name,external_id,name,long_desc,custom_fields
4421,height,"Umbrellas, Parasols & Shade Awnings",2.5 in.,bjs,3000000000003061763,Sol Armor 6' x 8' Cordless Outdoor Sun Shade with Valance,"If you are hot and uncomfortable in your outdoor spaces, the Sol Armor 6' x 8' Cordless Outdoor Sun Shade with Valance will allow you to enjoy your outdoor space again. Sol Armor shades are made from a high quality, durable PVC (poly vinyl chloride) fabric that will help cool your space, while allowing air to circulate. With up to 90% UV block, these shades reduce glare and keep your space cool. <BR><BR>These sun shades are raised/lowered using a detachable metal pole. By rolling your shades down before heat starts building up on your window, porch or patio, you can cool your space, lower energy costs, reduce blinding glare and fading of furniture. These shades also feature an aluminum, oil-rubbed bronze valance to protect your shade when not in use, extending the life of your fabric. The valance also allows for easy installation, as our mounting brackets can attach anywhere along the length of the valance. All mounting hardware, a bungee tie down system and an installation template is included.<BR><BR><SPAN style='font-weight:bold;'>Product Features:</SPAN><BR><BR><DIV><LI>Blocks the sun, not your view<LI>Pole and crank operated (comes with a detachable pole)<LI>Right or left side operation (default is right)<LI>Mounts to a wall or ceiling with included brackets<LI>Comes with a 3-sided aluminum valance to protect the shade while not in use<LI>Easy install brackets can be installed anywhere along the valance<LI>Up to 90% UV block<LI>Helps cool your space, lower energy costs, reduce blinding glare and fading of furniture<LI>All installation hardware and template included<LI>Available in Mohave, Humboldt, or Sedona<LI>Includes outdoor sun shade, installation hardware, and template<LI>Warranty: 2 years parts, 5 years fabric</DIV>","{'id': '3000000000003061761', 'Upc': '840017196163', 'club': [], 'name': 'Sol Armor 6' x 8' Cordless Outdoor Sun Shade with Valance', 'color': 'Humboldt', 'image': 's7d6.scene7.com/is/image/bjs/268030002', 'title': 'Sol Armor 6' x 8' Cordless Outdoor Sun Shade with Valance', 'mfname': 'Sol Armor', 'rating': [], 'buyable': 'buyable', 'channel': 'Desktop', 'club_id': [], 'pdplink': '?itemId=3000000000003061763', 'Includes': 'Outdoor Sun Shade, Installation Hardware, and Template', 'Keywords': '268030, Sol Armor, Roller shade, Outdoor roller shades, Roll up shade, Outdoor patio shades, Outdoor shade, Exterior shade, Solar shade, Sun shade, Outdoor blinds, Outdoor window shades, Roller blinds', 'Warranty': '2 Years Parts, 5 Years Fabric', 'keywords': ' ', 'map_type': [], 'mapprice': '0.00', 'maxPrice': '129.99', 'minPrice': '129.99', 'Shopbrand': 'Y', 'listRange': [], 'listprice': '129.99', 'Eaquantity': '1', 'Mount_type': 'Brackets', 'avg_rating': '0.0', 'partnumber': '268030', 'priceRange': '$129.99', 'Awning_type': 'Manual Retractable', 'Item_weight': '7 lbs.', 'catentry_id': '3000000000003061761', 'catgroup_id': [], 'define_attr': 'Color', 'maxMapPrice': '0.00', 'minMapPrice': '0.00', 'mobileimage': [], 'new_arrival': 'N', 'product_url': '/product/sol-armor-6-x-8-cordless-outdoor-sun-shade-with-valance/3000000000003061761', 'attribute_id': [], 'availability': ['online'], 'baseentry_id': '3000000000003061761', 'clubid_price': [], 'displayPrice': '129.99', 'gbi_toprated': '0', 'is_appliance': [], 'maxListPrice': '129.99', 'mfpartnumber': 'SA6MOH', 'minListPrice': '129.99', 'num_comments': '0', 'p_RecordType': 'Product', 'Free_shipping': 'N', 'giftcard_type': [], 'map_type_club': [], 'mapprice_club': [], 'product_price': '129.99', 'attribute_name': [], 'average_rating': '0.0', 'gbi_categories': [{'1': 'Patio & Outdoor Living', '2': 'Awnings & Outdoor Shading', '3': 'Window & Entryway Awnings', '4': [], '5': []}], 'gbi_has_rating': '1', 'gbi_promotions': [], 'instantSavings': [], 'Awning_m

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches,attri): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
    matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_{attri}-{today}.csv',index=False) 
looks_good(client, new_df,attri) 